In [1]:
import netCDF4
import numpy as np

* HGG: The HGG Product is the global merger of the HGS Products from all available satellites, where overlapping coverage is resolved in favor of the satellite with the best viewing geometry with a preference for geostationary results at lower latitudes and polar orbiter results in the polar regions. The time interval is 3 hr and the map grid is 1-degree-equivalent equal-area. This is the main ISCCP Cloud Product. 
* HGH: The HGH Product provides the monthly average of the HGG Product at each of eight times-of-day UTC. 
* HGM: The HGM Product is the average of the eight HGH Products for each month

** Remember to run jupyter-notebook from the project directory **

In [4]:
# read HGG files for a single day
# 8 files per day at 3 hour intervals
root_dir = 'ISCCP_HGG/'

# days = !ls $root_dir
# days = ['1991-06-01', '2001-06-01', '2011-06-01']
days = ['2011-06-01']
files_by_day = dict()
for d in days:
    hgg_files = !ls $root_dir$d
    files_by_day[d] = hgg_files

In [5]:
# labels = ['mean_cloud_amt_pct','mean_cloud_pres_cnt','mean_cloud_amt_ir_pct',
#           'mean_cloud_pres_ir_cnt','mean_surface_skin_temp_cnt', 'mean_surface_refl_cnt',
#          'near_surface_air_tmp_cnt','atmos_temp_prof_cnt','max_temp_cnt','tropos_temp_cnt','surface_pres_cnt',
#          'tropos_pres_cnt','near_surface_rel_hum_cnt','ozone_abund_cnt']
# features = ['cldamt','pc','cldamt_ir','pc_ir','ts','rs','airtemp','temp_profile','tmax','tmptab',
#             'psurf','ptrop','rh_nearsurf','ozone']


labels = ['snow_ice_pct','cloud_amount','mean_cloud_top_pres',
'mean_cloud_top_pres_ir','mean_cloud_top_temp','mean_cloud_top_temp_ir',
'mean_cloud_optic_depth','mean_cloud_optic_depth_ir']

features = ['snoice','cldamt','pc','pc_ir','tc','tc_ir','tau','tau_ir']

for d in files_by_day:
    print(d)
    for f in files_by_day[d]:
        fn = '%s%s/%s' % (root_dir, d, f)
        print(fn)
        # read the file
        ncfile = netCDF4.Dataset(fn, 'r')
        
        # get the timestamp
        date = "%s-%s-%s" % (ncfile.isccp_year, ncfile.isccp_month, ncfile.isccp_day)
        time = "%s:00" % ncfile.isccp_gmt
        
        # get number of sats
        num_sats = ncfile.isccp_number_of_satellites_contributing
        
        # get features
        file = open("%s.csv" % fn, "a")
        file.write('date,time,lat,lon,snow_ice_pct,cloud_amount,mean_cloud_top_pres,mean_cloud_top_pres_ir,mean_cloud_top_temp,mean_cloud_top_temp_ir,mean_cloud_optic_depth,mean_cloud_optic_depth_ir\n')

        lat_i = 0
        for lat in ncfile['lat']:
            #print(len(ncfile['lon']))
            lon_i = 0
            for lon in ncfile['lon']:
                if lon_i > 359:
                    print('breaking on lon_i=',lon_i)
                    break
                    
                dp = '%s,%s,%s,%s,' % (date,time,lat,lon)
                for fs in features:
                    v = ncfile[fs][0][lat_i][lon_i]
                    if isinstance(np.ma.min(v), np.ma.core.MaskedConstant):
                        v = -32767
                        #print('foo')

                    if fs == 'tau_ir':
                        dp = dp + '%f' % (v)
                    else:
                        # d = d + '%f,' % (v)
                        dp = dp + '%f,' % (v)
                # print(d)
                #print(fs,lat,lat_i,lon,lon_i)
                file.write('%s\n' % (dp))
                lon_i += 1
            lat_i += 1

        file.close()

2011-06-01
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.0000.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.0300.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.0600.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.0900.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.1200.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.1500.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.1800.GPC.10KM.CS00.EA1.00.nc
ISCCP_HGG/2011-06-01/ISCCP-Basic.HGG.v01r00.GLOBAL.2011.06.01.2100.GPC.10KM.CS00.EA1.00.nc


In [ ]:
days